In [ ]:
import csv
import pandas as pd
import json
import requests
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
# Get every single token and convert to pd dataframe

url = "https://api.coingecko.com/api/v3/coins/list"

headers = {"x-cg-pro-api-key": "CG-UHjW13FD2AW8Wxd4DiaJKJ1H"}

response = requests.get(url, headers=headers)
response_list = json.loads(response.text)

all_coins = pd.DataFrame(response_list)
all_coins['name'] = all_coins['name'].str.lower()
print(all_coins)

                        id symbol                 name
0                   01coin    zoc               01coin
1                   0chain    zcn                  zus
2      0-knowledge-network    0kn  0 knowledge network
3                    0-mee    ome                o-mee
4            0vix-protocol    vix        0vix protocol
...                    ...    ...                  ...
13491             zynecoin    zyn             zynecoin
13492              zynergy    zyn              zynergy
13493                zyrri    zyr                zyrri
13494                  zyx    zyx                  zyx
13495                  zzz    zzz          gosleep zzz

[13496 rows x 3 columns]


In [ ]:
# Get all token categories and convert names to id

coins = dict()
updated_coins = dict()

with open('token-categories.csv', 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        coins[row[0].strip().lower()] = row[1:]

coins['bitcoin'] = coins['\ufeffbitcoin']
del coins['\ufeffbitcoin']
print(coins)

for coin in coins:
    row = all_coins[all_coins['name'] == coin]
    if row.empty:
        row = all_coins[all_coins['id'] == coin]

    id = row.iloc[0]['id']
    updated_coins[id] = coins[coin]
    print(id, coin)

print(updated_coins)
print(coins['ordi'])


{'ethereum': ['Layer 1', 'PoS'], 'bnb': ['Exchange', 'Exchange'], 'solana': ['Layer 1', 'PoS'], 'xrp': ['Layer 1', 'Other'], 'cardano': ['Layer 1', 'PoS'], 'dogecoin': ['Currency', 'Memecoin'], 'avalanche': ['Layer 1', 'PoS'], 'shiba inu': ['Currency', 'Memecoin'], 'polkadot': ['Layer 1', 'PoS'], 'toncoin': ['Layer 1', 'PoS'], 'chainlink': ['Infrastructure', 'Oracle'], 'tron': ['Layer 1', 'PoS'], 'polygon': ['Layer 1', 'PoS'], 'uniswap': ['DeFi', 'DEX'], 'near protocol': ['Layer 1', 'PoS'], 'bitcoin cash': ['Currency', 'PoW'], 'litecoin': ['Currency', 'PoW'], 'internet computer': ['Layer 1', 'PoS'], 'aptos': ['Layer 1', 'PoS'], 'filecoin': ['Infrastructure', 'DePIN'], 'cosmos hub': ['Layer 1', 'PoS'], 'ethereum classic': ['Layer 1', 'Legacy'], 'immutable': ['Entertainment', 'Platform'], 'render': ['Infrastructure', 'DePIN'], 'hedera': ['Layer 1', 'PoS'], 'injective': ['Layer 1', 'PoS'], 'pepe': ['Currency', 'Memecoin'], 'stacks': ['Modular Layer', 'Execution'], 'optimism': ['Modular La

In [ ]:
# Get all coins from top100 list and construct dataframe of those with at least 1 appearance
# 247 coins that have had at least one appearance in the top 100

top100_df = pd.read_excel('top100.xlsx')
top100_df = top100_df.set_index('date_rebal_Q')
sums = top100_df.sum()
zerosum = sums[sums == 0].index

top100_df = top100_df.drop(zerosum, axis=1)
print(top100_df)

              bitcoin  ethereum  binancecoin  solana  cardano  ripple  \
date_rebal_Q                                                            
2020-01-01          1         1            1       0        1       1   
2020-04-01          1         1            1       0        1       1   
2020-07-01          1         1            1       1        1       1   
2020-10-01          1         1            1       1        1       1   
2021-01-01          1         1            1       1        1       1   
2021-04-01          1         1            1       1        1       1   
2021-07-01          1         1            1       1        1       1   
2021-10-01          1         1            1       1        1       1   
2022-01-01          1         1            1       1        1       1   
2022-04-01          1         1            1       1        1       1   
2022-07-01          1         1            1       1        1       1   
2022-10-01          1         1            1       

In [ ]:
categories = dict(set())
other = []

for coin in top100_df.columns:
    if coin in updated_coins:
        if updated_coins[coin][1] == 'Memecoin':
            if 'Memecoin' in categories:
                categories['Memecoin'].add(coin)
            else:
                categories['Memecoin'] = {coin}
        else:
            if updated_coins[coin][0] in categories:
                categories[updated_coins[coin][0]].add(coin)
            else:
                categories[updated_coins[coin][0]] = {coin}
    else:
        if 'n.a.' in categories:
            categories['n.a.'].add(coin)
        else:
            categories['n.a.'] = {coin}

#convert categories to csv
max_len = max(len(arr) for arr in categories.values())
categories_list = {key: list(value) + [None] * (max_len - len(value)) for key, value in categories.items()}
categories_df = pd.DataFrame(categories_list)
categories_df.to_csv('top100_categories.csv')

print(categories)
print(categories_df)

{'Currency': {'ravencoin', 'monero', 'decred', 'litecoin', 'bitcoin', 'ecash', 'verge', 'dash', 'zcash', 'bitcoin-cash-sv', 'bitcoin-cash', 'chia', 'zencash'}, 'Layer 1': {'quant-network', 'aptos', 'terra-luna-2', 'kadena', 'gnosis', 'flare-networks', 'eos', 'aelf', 'skale', 'celo', 'near', 'terra-luna', 'ripple', 'fantom', 'theta-token', 'ethereum-pow-iou', 'ocean-protocol', 'cardano', 'elrond-erd-2', 'kaspa', 'flow', 'oasis-network', 'tron', 'matic-network', 'kava', 'qtum', 'nem', 'xdce-crowd-sale', 'neo', 'tezos', 'injective-protocol', 'internet-computer', 'solana', 'casper-network', 'polkadot', 'kusama', 'waves', 'mina-protocol', 'oec-token', 'ethereum-classic', 'hedera-hashgraph', 'harmony', 'cosmos', 'sui', 'algorand', 'ontology', 'avalanche-2', 'digibyte', 'ethereum', 'nervos-network', 'theta-fuel', 'swipe', 'chiliz', 'iota', 'zilliqa', 'sei-network', 'icon', 'gas', 'astar', 'klay-token', 'vechain', 'stellar', 'aleph-zero', 'conflux-token', 'moonbeam', 'the-open-network'}, 'Exch

In [ ]:
# Create list of all coins
top100 = set()
for category in categories.values():
    top100 = top100.union(category)

print(top100)

{'xido-finance', 'celestia', 'ark', 'nuls', 'zcash', 'pnetwork', 'kucoin-shares', 'bitcoin-gold', 'golem', 'dent', '1inch', 'spell-token', 'vethor-token', 'ravencoin', 'nucypher', 'band-protocol', 'deso', 'mantle', 'maker', 'osmosis', 'blockstack', 'iexec-rlc', 'axie-infinity', 'flow', 'bytom', 'smooth-love-potion', 'nano', 'worldcoin-wld', 'fsn', 'curve-dao-token', 'serum', 'internet-computer', 'chainlink', 'waltonchain', 'polkadot', 'troy', 'contentos', 'audius', 'rif-token', 'ecomi', 'zencash', 'lto-network', 'swftcoin', 'ontology', 'ethereum', 'nervos-network', 'litecoin', 'celsius-degree-token', 'ecash', 'sei-network', 'gmx', 'filecoin', 'huobi-token', 'secret', 'yfii-finance', 'bittensor', 'klay-token', 'just', 'loopring', 'aleph-zero', 'beldex', 'bitcoin-diamond', 'origin-protocol', 'blur', 'shiba-inu', 'v-systems', 'bitcoin', 'quant-network', 'safepal', 'nkn', 'axelar', 'eos', 'havven', 'fetch-ai', 'skale', 'helium', 'celo', 'near', 'binancecoin', 'ethereum-pow-iou', 'theta-tok

In [ ]:
# Create df for all and each catagory (10 categories + 1 for All)

all_coin_price = pd.read_csv('prices.csv')
all_coin_price.rename(columns={all_coin_price.columns[0] : 'Date'}, inplace=True)
all_coin_price['Date'] = pd.to_datetime(all_coin_price['Date'])
all_coin_price.set_index('Date', inplace=True)

top_coins_data = all_coin_price[all_coin_price.columns.intersection(top100)]

cat_dfs = {}
for category, values in categories.items():
    cat_dfs[category] = top_coins_data[top_coins_data.columns.intersection(values)]

cat_dfs['All'] = top_coins_data

# convert top coins to csv
top_coins_data.to_csv('top100_price_data.csv')

print(cat_dfs['Memecoin'])

            shiba-inu  dogecoin   ordinals      pepe  baby-doge-coin      bonk
Date                                                                          
2013-01-01        NaN       NaN        NaN       NaN             NaN       NaN
2013-04-28        NaN       NaN        NaN       NaN             NaN       NaN
2013-04-29        NaN       NaN        NaN       NaN             NaN       NaN
2013-04-30        NaN       NaN        NaN       NaN             NaN       NaN
2013-05-01        NaN       NaN        NaN       NaN             NaN       NaN
...               ...       ...        ...       ...             ...       ...
2024-03-03   0.000023  0.142764  80.510616  0.000004    2.067940e-09  0.000024
2024-03-04   0.000023  0.153913  81.354912  0.000006    1.997608e-09  0.000027
2024-03-05   0.000035  0.182485  87.123796  0.000007    2.533100e-09  0.000038
2024-03-06   0.000036  0.155775  76.168225  0.000006    2.141423e-09  0.000031
2024-03-07   0.000034  0.158267  75.510210  0.000007

In [ ]:
# Get price at end of each week

df_weekly_price = {}
for category, df in cat_dfs.items():
    weekly = df.resample('W').last()
    df_weekly_price[category] = weekly

print(df_weekly_price['Memecoin'])

            shiba-inu  dogecoin   ordinals      pepe  baby-doge-coin      bonk
Date                                                                          
2013-01-06        NaN       NaN        NaN       NaN             NaN       NaN
2013-01-13        NaN       NaN        NaN       NaN             NaN       NaN
2013-01-20        NaN       NaN        NaN       NaN             NaN       NaN
2013-01-27        NaN       NaN        NaN       NaN             NaN       NaN
2013-02-03        NaN       NaN        NaN       NaN             NaN       NaN
...               ...       ...        ...       ...             ...       ...
2024-02-11   0.000009  0.081365  61.610343  0.000001    1.201164e-09  0.000013
2024-02-18   0.000010  0.083604  68.176413  0.000001    1.190718e-09  0.000013
2024-02-25   0.000010  0.086104  63.979568  0.000001    1.158053e-09  0.000012
2024-03-03   0.000023  0.142764  80.510616  0.000004    2.067940e-09  0.000024
2024-03-10   0.000034  0.158267  75.510210  0.000007

In [ ]:
# Create df for all categories with cumulative, weekly, and daily returns

df_cat_ret = {}
df_cat_daily = {}
df_cat_weekly = {}

for key, df in cat_dfs.items():

    df_returns = df.pct_change()
    df_returns = df_returns.fillna(0)

    df_returns_pct = df.pct_change() * 100
    df_returns_pct = df_returns_pct.fillna(0)
    df_cat_daily[key] = df_returns_pct

    df_cumulative = (1 + df_returns).cumprod() - 1
    df_cat_ret[key] = df_cumulative

for category, df in df_weekly_price.items():

    df_weekly = df.pct_change() * 100
    df_cat_weekly[category] = df_weekly

#print(df_cat_ret['All'])
print(df_cat_weekly['Memecoin']) # pct change between weekly prices

for category, df in df_cat_weekly.items():
    print(f'{category}: {len(df.columns)}')

             shiba-inu   dogecoin   ordinals        pepe  baby-doge-coin  \
Date                                                                       
2013-01-06         NaN        NaN        NaN         NaN             NaN   
2013-01-13         NaN        NaN        NaN         NaN             NaN   
2013-01-20         NaN        NaN        NaN         NaN             NaN   
2013-01-27         NaN        NaN        NaN         NaN             NaN   
2013-02-03         NaN        NaN        NaN         NaN             NaN   
...                ...        ...        ...         ...             ...   
2024-02-11    3.407341   3.375468  11.993628    9.647493        7.494681   
2024-02-18    2.343734   2.751257  10.657415   16.963975       -0.869624   
2024-02-25   -0.013999   2.990240  -6.155861   10.332550       -2.743304   
2024-03-03  135.788432  65.805228  25.838012  234.935626       78.570323   
2024-03-10   47.961879  10.858650  -6.210865   65.237171        9.644701   

           

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:59: RuntimeWarning: invalid value encountered in accumulate
  return bound(*args, **kwds)


In [ ]:
# Create df for all categories with average cumulative and daily returns

df_cat_avg = {}
df_cat_avg_d = {}
df_cat_avg_w = {}

for key, df in df_cat_ret.items():

    df_avg = df.mean(axis=1)
    df_avg = pd.DataFrame(df_avg, columns=[key])
    df_cat_avg[key] = df_avg

for key, df in df_cat_daily.items():

    df_avg = df.mean(axis=1)
    df_avg = pd.DataFrame(df_avg, columns=[key])
    df_cat_avg_d[key] = df_avg

for key, df in df_cat_weekly.items():

    df_avg = df.mean(axis=1)
    df_avg = pd.DataFrame(df_avg, columns=[key])
    df_cat_avg_w[key] = df_avg

print(df_cat_avg_w['All']) # avg of category for pct change between weekly prices

                  All
Date                 
2013-01-06        NaN
2013-01-13        NaN
2013-01-20        NaN
2013-01-27        NaN
2013-02-03        NaN
...               ...
2024-02-11   6.616507
2024-02-18  10.148962
2024-02-25   5.564545
2024-03-03  23.313192
2024-03-10   4.349575

[584 rows x 1 columns]


In [ ]:
# Create df with all categories containing average cumulative returns

df_combined_avg = pd.concat([df for df in df_cat_avg.values()], axis=1)
df_combined_avg_d = pd.concat([df for df in df_cat_avg_d.values()], axis=1)
df_combined_avg_w = pd.concat([df for df in df_cat_avg_w.values()], axis=1)

print(df_combined_avg_w) # all averages of pct change (weekly)

             Currency    Layer 1   Exchange    Memecoin       DeFi  \
Date                                                                 
2013-01-06        NaN        NaN        NaN         NaN        NaN   
2013-01-13        NaN        NaN        NaN         NaN        NaN   
2013-01-20        NaN        NaN        NaN         NaN        NaN   
2013-01-27        NaN        NaN        NaN         NaN        NaN   
2013-02-03        NaN        NaN        NaN         NaN        NaN   
...               ...        ...        ...         ...        ...   
2024-02-11   1.792946   6.266194   5.420402   10.765668   4.906380   
2024-02-18  10.233316  10.080452   5.322218    4.690049   5.318102   
2024-02-25   2.731009   3.366065   8.130128   -0.725671  10.007258   
2024-03-03  36.848075  22.862783  16.478604  108.209304  12.247419   
2024-03-10  -1.318900   3.762561   4.285395   25.155980   4.747202   

            Infrastructure  Modular Layer       n.a.  Entertainment  \
Date              

Identifying weeks in which memecoins outperformed every other sector. Additionally, find the weeks that every other sector outperformed in order to compare memecoin outperformance to other coin's.

In [ ]:
# Compare Memecoins to other sectors (44 weeks from 2020 to present)

temp = df_combined_avg_w.drop('All', axis=1)
other_columns = temp.columns[temp.columns != 'Memecoin']
df_memecoin_top_all = temp[temp['Memecoin'] > temp[other_columns].max(axis=1)]
df_memecoin_top_2020 = df_memecoin_top_all[df_memecoin_top_all.index.year >= 2020]

# convert top coins to csv
df_memecoin_top_2020.to_csv('memecoin_topsector.csv')

print(df_memecoin_top_2020)
print(len(df_memecoin_top_2020))

             Currency    Layer 1   Exchange     Memecoin       DeFi  \
Date                                                                  
2020-03-15 -46.451609 -48.460716 -43.172554   -31.766998 -44.670071   
2020-04-12   2.185459   1.831958  -0.940869     6.504668   2.356162   
2020-05-03   8.398340  11.396377   7.693048    19.528950  19.336724   
2020-07-12   6.772911  13.220715   4.896181    64.665834   2.909641   
2020-09-06 -21.229107 -24.565711 -13.981598    -1.125063 -23.206199   
2020-09-20  -4.646132  -4.006552  -1.759679    24.618608 -16.081240   
2020-12-27   0.102048  -5.057679  -7.692556     5.743233 -10.791010   
2021-01-03   1.162852   5.485289   6.190789    79.371708  12.859388   
2021-01-31  10.058808  11.706711  12.373248  2086.286057  42.578576   
2021-02-07  15.676243  24.394940  53.347070    83.257525  33.406254   
2021-03-21   0.768316   7.873896   4.124156    86.999269   0.261959   
2021-04-18  31.711630  10.594314  -0.372172  1296.064189   9.576731   
2021-0

In [ ]:
# Get other sector max

top_stats = dict()
for coin in temp.columns:
    other_columns = temp.columns[temp.columns != coin]
    df_coin_top = temp[temp[coin] > temp[other_columns].max(axis=1)]
    df_coin_top_2020 = df_coin_top[df_coin_top.index.year >= 2020]
    print(coin, len(df_coin_top_2020))
    top_stats[coin] = len(df_coin_top_2020)

print(top_stats)

Currency 17
Layer 1 7
Exchange 23
Memecoin 47
DeFi 19
Infrastructure 15
Modular Layer 23
n.a. 11
Entertainment 23
Other 34
{'Currency': 17, 'Layer 1': 7, 'Exchange': 23, 'Memecoin': 47, 'DeFi': 19, 'Infrastructure': 15, 'Modular Layer': 23, 'n.a.': 11, 'Entertainment': 23, 'Other': 34}


In [ ]:
#Pie Graph for top weeks since 2020

cats_top_labels = list(top_stats.keys())
cats_top_vals = list(top_stats.values())
#percentages = [(val/sum(cats_top_vals)) * 100 for val in cats_top_vals]
text_vals = [f'{val} weeks' for val in cats_top_vals]

trace = go.Pie(labels=cats_top_labels, values=cats_top_vals, text=text_vals)
layout = go.Layout(title='Weeks as Top Performing Sector by Category', legend=dict(orientation='h', yanchor='bottom', y=-5), margin=dict(b=90))
top_perf_graph = go.Figure(data=[trace], layout=layout)

top_perf_graph.show()

Here we relate memecoin outperformance to the cyrpto market performance.

We identify weeks that memecoins had a higher weekly pct change than the market as a whole.

These are broken into 16 groups:

*   MC PC greater than Market PC -> 4/1 weeks after, all time / 2020-
*   MC PC at least 10% greater than Market PC -> 4/1 weeks after, all time / 2020-
*   MC PC at least 20% greater than Market PC -> 4/1 weeks after, all time / 2020-
*   MC PC at least 50% greater than Market PC -> 4/1 weeks after, all time / 2020-

All categories have their mean and median >0



In [ ]:
# Get weeks that each sector is outperformed

meme_outperformance_sec = {}

for cat, df in df_cat_avg_w.items():
    if cat != 'Memecoin':
        df_top = pd.DataFrame(df[df[cat] > df_cat_avg_w['Memecoin']['Memecoin']][cat])
        meme_outperformance_sec[cat] = df_top

print(meme_outperformance_sec['All']) #weeks that mc outperformed market
print(df_cat_avg_w['Memecoin'])

                  All
Date                 
2013-12-29  18.083909
2014-01-05  -0.363393
2014-02-02  -3.534150
2014-02-23  33.430578
2014-03-02 -11.928168
...               ...
2024-01-21  -5.072636
2024-01-28  -1.198513
2024-02-04  -2.303120
2024-02-18  10.148962
2024-02-25   5.564545

[309 rows x 1 columns]
              Memecoin
Date                  
2013-01-06         NaN
2013-01-13         NaN
2013-01-20         NaN
2013-01-27         NaN
2013-02-03         NaN
...                ...
2024-02-11   10.765668
2024-02-18    4.690049
2024-02-25   -0.725671
2024-03-03  108.209304
2024-03-10   25.155980

[584 rows x 1 columns]


In [ ]:
# View market 4 week after Memecoin out performance

market_under = meme_outperformance_sec['All']
market_all = df_combined_avg_w['All']
market_under_2020 = market_under[market_under.index.year >= 2020]

# all time
four_weeks_after = []
for date in market_under.index.tolist():
    next_four = market_all[(market_all.index > date) & (market_all.index <= date + pd.DateOffset(weeks=4))]
    next_four = next_four.to_frame()
    next_four['Cumulative'] = next_four['All'].cumsum()
    four_weeks_after.append(next_four)

four_weeks_cum = []
for data in four_weeks_after:
    four_weeks_cum.append(data.iloc[-1, -1])

mean = sum(four_weeks_cum) / len(four_weeks_cum)
median = sorted(four_weeks_cum)[len(four_weeks_cum)//2]
count = len(four_weeks_cum)
max_ = max(four_weeks_cum)
min_ = min(four_weeks_cum)

four_weeks_return_graph = px.box(x=four_weeks_cum, points='all')
four_weeks_return_graph.update_layout(title='Cumulative Returns 4 Weeks after MC outperformance', xaxis_title='Cumulative Returns (%)')
four_weeks_return_graph.show()
print(f'mean: {mean}, median: {median}, count: {count}\nmax: {max_}, min: {min_}')


# 2020-presentfour_weeks_after = []
four_weeks_after = []
for date in market_under_2020.index.tolist():
    next_four = market_all[(market_all.index > date) & (market_all.index <= date + pd.DateOffset(weeks=4))]
    next_four = next_four.to_frame()
    next_four['Cumulative'] = next_four['All'].cumsum()
    four_weeks_after.append(next_four)

four_weeks_cum = []
for data in four_weeks_after:
    four_weeks_cum.append(data.iloc[-1, -1])

mean = sum(four_weeks_cum) / len(four_weeks_cum)
median = sorted(four_weeks_cum)[len(four_weeks_cum)//2]
count = len(four_weeks_cum)
max_ = max(four_weeks_cum)
min_ = min(four_weeks_cum)

four_weeks_return_graph = px.box(x=four_weeks_cum, points='all')
four_weeks_return_graph.update_layout(title='Cumulative Returns 4 Weeks after MC outperformance (2020-)', xaxis_title='Cumulative Returns (%)')
four_weeks_return_graph.show()
print(f'mean: {mean}, median: {median}, count: {count}\nmax: {max_}, min: {min_}')


mean: 14.51893198220407, median: 6.637637361913303, count: 309
max: 245.41421131517154, min: -60.988566390176985


mean: 13.476020086653014, median: 6.691726172699275, count: 126
max: 245.41421131517154, min: -55.22836128750579


In [ ]:
# View market 1 week after Memecoin out performance

# all time
one_week_after = []
for date in market_under.index.tolist():
    next_week = market_all.loc[date + pd.DateOffset(weeks=1)]
    one_week_after.append(next_week)

mean = sum(one_week_after) / len(one_week_after)
median = sorted(one_week_after)[len(one_week_after)//2]
count = len(one_week_after)
max_ = max(one_week_after)
min_ = min(one_week_after)

one_week_return_graph = px.box(x=one_week_after, points='all')
one_week_return_graph.update_layout(title='Cumulative Returns 1 Week after MC outperformance', xaxis_title='Cumulative Returns (%)')
one_week_return_graph.show()
print(f'mean: {mean}, median: {median}, count: {count}\nmax: {max_}, min: {min_}')

# 2020-present
one_week_after = []
for date in market_under_2020.index.tolist():
    next_week = market_all.loc[date + pd.DateOffset(weeks=1)]
    one_week_after.append(next_week)

mean = sum(one_week_after) / len(one_week_after)
median = sorted(one_week_after)[len(one_week_after)//2]
count = len(one_week_after)
max_ = max(one_week_after)
min_ = min(one_week_after)

one_week_return_graph = px.box(x=one_week_after, points='all')
one_week_return_graph.update_layout(title='Cumulative Returns 1 Week after MC outperformance (2020-)', xaxis_title='Cumulative Returns (%)')
one_week_return_graph.show()
print(f'mean: {mean}, median: {median}, count: {count}\nmax: {max_}, min: {min_}')

mean: 3.18301706101251, median: 2.156198629806471, count: 309
max: 107.72476311944261, min: -47.464013651531715


mean: 2.5295725493135515, median: 3.264091566485805, count: 126
max: 33.99739124956111, min: -47.464013651531715


In [ ]:
# Market 4 weeks after >=10% outperformance

df_market = df_combined_avg_w['All']
df_meme = df_combined_avg_w['Memecoin']
df_both = pd.concat([df_market, df_meme], axis=1)
df_meme_10 = df_market[df_both['Memecoin'] >= (df_both['All'] + 10)].to_frame()

df_market_2020 = df_market[df_market.index.year >= 2020]
df_both_2020 = df_both[df_both.index.year >= 2020]
df_meme_10_2020 = df_market_2020[df_both_2020['Memecoin'] >= (df_both_2020['All'] + 10)].to_frame()


# all time
four_after_10 = []
for date in df_meme_10.index.tolist():
    next_four = market_all[(market_all.index > date) & (market_all.index <= date + pd.DateOffset(weeks=4))]
    next_four = next_four.to_frame()
    next_four['Cumulative'] = next_four['All'].cumsum()
    if next_four.size != 0:
        four_after_10.append(next_four)


four_10_cum = []
for data in four_after_10:
    four_10_cum.append(data.iloc[-1, -1])

mean = sum(four_10_cum) / len(four_10_cum)
median = sorted(four_10_cum)[len(four_10_cum)//2]
count = len(four_10_cum)
max_ = max(four_10_cum)
min_ = min(four_10_cum)


four_10_return_graph = px.box(x=four_10_cum, points='all')
four_10_return_graph.update_layout(title='Cumulative Returns 4 Weeks after MC Outperformance', xaxis_title='Cumulative Returns (%)')
four_10_return_graph.add_annotation(text='When MC Outperforms by at least 10%', showarrow=False)
four_10_return_graph.show()
print(f'mean: {mean}, median: {median}, count: {count}\nmax: {max_}, min: {min_}')

# 2020-present
four_after_10 = []
for date in df_meme_10_2020.index.tolist():
    next_four = market_all[(market_all.index > date) & (market_all.index <= date + pd.DateOffset(weeks=4))]
    next_four = next_four.to_frame()
    next_four['Cumulative'] = next_four['All'].cumsum()
    if next_four.size != 0:
        four_after_10.append(next_four)


four_10_cum = []
for data in four_after_10:
    four_10_cum.append(data.iloc[-1, -1])

mean = sum(four_10_cum) / len(four_10_cum)
median = sorted(four_10_cum)[len(four_10_cum)//2]
count = len(four_10_cum)
max_ = max(four_10_cum)
min_ = min(four_10_cum)


four_10_return_graph = px.box(x=four_10_cum, points='all')
four_10_return_graph.update_layout(title='Cumulative Returns 4 Weeks after MC Outperformance (2020-)', xaxis_title='Cumulative Returns (%)')
four_10_return_graph.add_annotation(text='When MC Outperforms by at least 10%', showarrow=False)
four_10_return_graph.show()
print(f'mean: {mean}, median: {median}, count: {count}\nmax: {max_}, min: {min_}')

mean: 26.017611969370176, median: 15.551891565211433, count: 75
max: 223.46782216416213, min: -57.432598314869516


mean: 24.655209395776357, median: 13.672364932178033, count: 38
max: 223.46782216416213, min: -44.67293902921706


In [ ]:
# Market 1 week after >=10% outperformance

# all time
one_after_10 = []
for date in df_meme_10.index.tolist():
    next = date + pd.DateOffset(weeks=1)
    try:
        next_week = market_all.loc[date + pd.DateOffset(weeks=1)]
        one_after_10.append(next_week)
    except KeyError:
        continue

mean = sum(one_after_10) / len(one_after_10)
median = sorted(one_after_10)[len(one_after_10)//2]
count = len(one_after_10)
max_ = max(one_after_10)
min_ = min(one_after_10)

one_10_return_graph = px.box(x=one_after_10, points='all')
one_10_return_graph.update_layout(title='Cumulative Returns 1 Week after MC outperformance', xaxis_title='Cumulative Returns (%)')
one_10_return_graph.add_annotation(text='When MC Outperforms by at least 10%', showarrow=False)
one_10_return_graph.show()
print(f'mean: {mean}, median: {median}, count: {count}\nmax: {max_}, min: {min_}')

# 2020-present
one_after_10 = []
for date in df_meme_10_2020.index.tolist():
    next = date + pd.DateOffset(weeks=1)
    try:
        next_week = market_all.loc[date + pd.DateOffset(weeks=1)]
        one_after_10.append(next_week)
    except KeyError:
        continue

mean = sum(one_after_10) / len(one_after_10)
median = sorted(one_after_10)[len(one_after_10)//2]
count = len(one_after_10)
max_ = max(one_after_10)
min_ = min(one_after_10)

one_10_return_graph = px.box(x=one_after_10, points='all')
one_10_return_graph.update_layout(title='Cumulative Returns 1 Week after MC outperformance (2020-)', xaxis_title='Cumulative Returns (%)')
one_10_return_graph.add_annotation(text='When MC Outperforms by at least 10%', showarrow=False)
one_10_return_graph.show()
print(f'mean: {mean}, median: {median}, count: {count}\nmax: {max_}, min: {min_}')

mean: 9.862147142250771, median: 5.4710484053364254, count: 75
max: 252.08673584016222, min: -31.058515015100824


mean: 12.778647562624696, median: 6.050235947256514, count: 38
max: 252.08673584016222, min: -31.058515015100824


In [ ]:
# Market 4 weeks after >=20% outperformance

df_meme_20 = df_market[df_both['Memecoin'] >= (df_both['All'] + 20)].to_frame()
df_meme_20_2020 = df_market_2020[df_both_2020['Memecoin'] >= (df_both_2020['All'] + 20)].to_frame()

# all time
four_after_20 = []
for date in df_meme_20.index.tolist():
    next_four = market_all[(market_all.index > date) & (market_all.index <= date + pd.DateOffset(weeks=4))]
    next_four = next_four.to_frame()
    next_four['Cumulative'] = next_four['All'].cumsum()
    if next_four.size != 0:
        four_after_20.append(next_four)

four_20_cum = []
for data in four_after_20:
    four_20_cum.append(data.iloc[-1, -1])

mean = sum(four_20_cum) / len(four_20_cum)
median = sorted(four_20_cum)[len(four_20_cum)//2]
count = len(four_20_cum)
max_ = max(four_20_cum)
min_ = min(four_20_cum)


four_20_return_graph = px.box(x=four_20_cum, points='all')
four_20_return_graph.update_layout(title='Cumulative Returns 4 Weeks after MC Outperformance', xaxis_title='Cumulative Returns (%)')
four_20_return_graph.add_annotation(text='When MC Outperforms by at least 20%', showarrow=False)
four_20_return_graph.show()
print(f'mean: {mean}, median: {median}, count: {count}\nmax: {max_}, min: {min_}')

# 2020 - present
four_after_20 = []
for date in df_meme_20_2020.index.tolist():
    next_four = market_all[(market_all.index > date) & (market_all.index <= date + pd.DateOffset(weeks=4))]
    next_four = next_four.to_frame()
    next_four['Cumulative'] = next_four['All'].cumsum()
    if next_four.size != 0:
        four_after_20.append(next_four)

four_20_cum = []
for data in four_after_20:
    four_20_cum.append(data.iloc[-1, -1])

mean = sum(four_20_cum) / len(four_20_cum)
median = sorted(four_20_cum)[len(four_20_cum)//2]
count = len(four_20_cum)
max_ = max(four_20_cum)
min_ = min(four_20_cum)


four_20_return_graph = px.box(x=four_20_cum, points='all')
four_20_return_graph.update_layout(title='Cumulative Returns 4 Weeks after MC Outperformance (2020-)', xaxis_title='Cumulative Returns (%)')
four_20_return_graph.add_annotation(text='When MC Outperforms by at least 20%', showarrow=False)
four_20_return_graph.show()
print(f'mean: {mean}, median: {median}, count: {count}\nmax: {max_}, min: {min_}')

mean: 31.041187165699924, median: 18.720592277186277, count: 44
max: 223.46782216416213, min: -48.713830069454644


mean: 30.962565739045225, median: 18.720592277186277, count: 26
max: 223.46782216416213, min: -44.67293902921706


In [ ]:
# Market 1 week after >=20% outperformance

# all time
one_after_20 = []
for date in df_meme_20.index.tolist():
    next = date + pd.DateOffset(weeks=1)
    try:
        next_week = market_all.loc[date + pd.DateOffset(weeks=1)]
        one_after_20.append(next_week)
    except KeyError:
        continue

mean = sum(one_after_20) / len(one_after_20)
median = sorted(one_after_20)[len(one_after_20)//2]
count = len(one_after_20)
max_ = max(one_after_20)
min_ = min(one_after_20)

one_20_return_graph = px.box(x=one_after_20, points='all')
one_20_return_graph.update_layout(title='Cumulative Returns 1 Week after MC outperformance', xaxis_title='Cumulative Returns (%)')
one_20_return_graph.add_annotation(text='When MC Outperforms by at least 20%', showarrow=False)
one_20_return_graph.show()
print(f'mean: {mean}, median: {median}, count: {count}\nmax: {max_}, min: {min_}')

# 2020 - present
one_after_20 = []
for date in df_meme_20_2020.index.tolist():
    next = date + pd.DateOffset(weeks=1)
    try:
        next_week = market_all.loc[date + pd.DateOffset(weeks=1)]
        one_after_20.append(next_week)
    except KeyError:
        continue

mean = sum(one_after_20) / len(one_after_20)
median = sorted(one_after_20)[len(one_after_20)//2]
count = len(one_after_20)
max_ = max(one_after_20)
min_ = min(one_after_20)

one_20_return_graph = px.box(x=one_after_20, points='all')
one_20_return_graph.update_layout(title='Cumulative Returns 1 Week after MC outperformance (2020-)', xaxis_title='Cumulative Returns (%)')
one_20_return_graph.add_annotation(text='When MC Outperforms by at least 20%', showarrow=False)
one_20_return_graph.show()
print(f'mean: {mean}, median: {median}, count: {count}\nmax: {max_}, min: {min_}')

mean: 13.419944862483703, median: 6.050235947256514, count: 44
max: 252.08673584016222, min: -31.058515015100824


mean: 17.043346553080468, median: 6.050235947256514, count: 26
max: 252.08673584016222, min: -31.058515015100824


In [ ]:
# Market 4 weeks after >=50% outperformance

df_meme_50 = df_market[df_both['Memecoin'] >= (df_both['All'] + 50)].to_frame()
df_meme_50_2020 = df_market_2020[df_both_2020['Memecoin'] >= (df_both_2020['All'] + 50)].to_frame()


# all time
four_after_50 = []
for date in df_meme_50.index.tolist():
    next_four = market_all[(market_all.index > date) & (market_all.index <= date + pd.DateOffset(weeks=4))]
    next_four = next_four.to_frame()
    next_four['Cumulative'] = next_four['All'].cumsum()
    if next_four.size != 0:
        four_after_50.append(next_four)

four_50_cum = []
for data in four_after_50:
    four_50_cum.append(data.iloc[-1, -1])

mean = sum(four_50_cum) / len(four_50_cum)
median = sorted(four_50_cum)[len(four_50_cum)//2]
count = len(four_50_cum)
max_ = max(four_50_cum)
min_ = min(four_50_cum)


four_50_return_graph = px.box(x=four_50_cum, points='all')
four_50_return_graph.update_layout(title='Cumulative Returns 4 Weeks after MC Outperformance', xaxis_title='Cumulative Returns (%)')
four_50_return_graph.add_annotation(text='When MC Outperforms by at least 50%', showarrow=False)
four_50_return_graph.show()
print(f'mean: {mean}, median: {median}, count: {count}\nmax: {max_}, min: {min_}')

# 2020 - present
four_after_50 = []
for date in df_meme_50_2020.index.tolist():
    next_four = market_all[(market_all.index > date) & (market_all.index <= date + pd.DateOffset(weeks=4))]
    next_four = next_four.to_frame()
    next_four['Cumulative'] = next_four['All'].cumsum()
    if next_four.size != 0:
        four_after_50.append(next_four)

four_50_cum = []
for data in four_after_50:
    four_50_cum.append(data.iloc[-1, -1])

mean = sum(four_50_cum) / len(four_50_cum)
median = sorted(four_50_cum)[len(four_50_cum)//2]
count = len(four_50_cum)
max_ = max(four_50_cum)
min_ = min(four_50_cum)


four_50_return_graph = px.box(x=four_50_cum, points='all')
four_50_return_graph.update_layout(title='Cumulative Returns 4 Weeks after MC Outperformance (2020-)', xaxis_title='Cumulative Returns (%)')
four_50_return_graph.add_annotation(text='When MC Outperforms by at least 50%', showarrow=False)
four_50_return_graph.show()
print(f'mean: {mean}, median: {median}, count: {count}\nmax: {max_}, min: {min_}')

mean: 24.952301185158593, median: 18.720592277186277, count: 16
max: 100.55923851975932, min: -44.67293902921706


mean: 32.750766426973136, median: 23.816318073794477, count: 13
max: 100.55923851975932, min: -44.67293902921706


In [ ]:
# Market 1 week after >=50% outperformance


# all time
one_after_50 = []
for date in df_meme_50.index.tolist():
    next = date + pd.DateOffset(weeks=1)
    try:
        next_week = market_all.loc[date + pd.DateOffset(weeks=1)]
        one_after_50.append(next_week)
    except KeyError:
        continue

mean = sum(one_after_50) / len(one_after_50)
median = sorted(one_after_50)[len(one_after_50)//2]
count = len(one_after_50)
max_ = max(one_after_50)
min_ = min(one_after_50)

one_20_return_graph = px.box(x=one_after_50, points='all')
one_20_return_graph.update_layout(title='Cumulative Returns 1 Week after MC outperformance', xaxis_title='Cumulative Returns (%)')
one_20_return_graph.add_annotation(text='When MC Outperforms by at least 50%', showarrow=False)
one_20_return_graph.show()
print(f'mean: {mean}, median: {median}, count: {count}\nmax: {max_}, min: {min_}')


# 2020 - present
one_after_50 = []
for date in df_meme_50_2020.index.tolist():
    next = date + pd.DateOffset(weeks=1)
    try:
        next_week = market_all.loc[date + pd.DateOffset(weeks=1)]
        one_after_50.append(next_week)
    except KeyError:
        continue

mean = sum(one_after_50) / len(one_after_50)
median = sorted(one_after_50)[len(one_after_50)//2]
count = len(one_after_50)
max_ = max(one_after_50)
min_ = min(one_after_50)

one_20_return_graph = px.box(x=one_after_50, points='all')
one_20_return_graph.update_layout(title='Cumulative Returns 1 Week after MC outperformance (2020-)', xaxis_title='Cumulative Returns (%)')
one_20_return_graph.add_annotation(text='When MC Outperforms by at least 50%', showarrow=False)
one_20_return_graph.show()
print(f'mean: {mean}, median: {median}, count: {count}\nmax: {max_}, min: {min_}')

mean: 7.0294202123433, median: 4.3495753276953035, count: 16
max: 62.43654914794507, min: -27.450568269682538


mean: 11.615260723046415, median: 6.050235947256514, count: 13
max: 62.43654914794507, min: -27.450568269682538


Here are a variety of graphs to show stats n whatnot

In [ ]:
# Graphs

#Cumulative Returns
trace_meme = go.Scatter(x=df_combined_avg_d.index, y=df_combined_avg['Memecoin'], mode='lines', name='Memecoin')
trace_all = go.Scatter(x=df_combined_avg_d.index, y=df_combined_avg['All'], mode='lines', name='Market')

cum_graph = go.Figure([trace_meme, trace_all])
cum_graph.update_layout(title='Historical Cumulative Returns',
                  xaxis_title='Date',
                  yaxis_title='Cumulative Returns (%)')
#cum_graph.show()

#Daily Returns
trace_meme_d = go.Scatter(x=df_combined_avg_d.index, y=df_combined_avg_d['Memecoin'], mode='lines', name='Memecoin')
trace_all_d = go.Scatter(x=df_combined_avg_d.index, y=df_combined_avg_d['All'], mode='lines', name='Market')

daily_graph = go.Figure([trace_meme_d, trace_all_d])
daily_graph.update_layout(title='Historical Daily Returns',
                        xaxis_title='Date',
                        yaxis_title='Daily Returns (%)',
                        yaxis=dict(range=[-50, 100]),
                        xaxis=dict(range=['2020-01-01', '2024-03-07']))
#daily_graph.show()

#Weekly Returns
trace_meme_w = go.Scatter(x=df_combined_avg_w.index, y=df_combined_avg_w['Memecoin'], mode='lines', name='Memecoin')
trace_all_w = go.Scatter(x=df_combined_avg_w.index, y=df_combined_avg_w['All'], mode='lines', name='Market')

weekly_graph = go.Figure([trace_meme_w, trace_all_w])
weekly_graph.update_layout(title='Historical Weekly Returns',
                        xaxis_title='Date',
                        yaxis_title='Weekly Returns (%)',
                        yaxis=dict(range=[-50, 100]),
                        xaxis=dict(range=['2020-01-01', '2024-03-07']))
weekly_graph.show()

#Weeks of Memecoin top performance
traces = []
for column in df_memecoin_top_2020.columns:
    trace = go.Scatter(x=df_memecoin_top_2020.index, y=df_memecoin_top_2020[column], mode='lines+markers', name=column)
    traces.append(trace)

memetop_graph = go.Figure(traces)
memetop_graph.update_layout(title='Weeks with Memecoins as Top Sector',
                        xaxis_title='Date',
                        yaxis_title='Weekly Returns (%)',
                        yaxis=dict(range=[-60, 100]))

memetop_graph.show()

#Weeks of Memecoin top performance
df_temp_meme = df_cat_avg_w['Memecoin']
df_temp_all = meme_outperformance_sec['All']
trace_meme_w = go.Scatter(x=df_temp_meme.index, y=df_temp_meme['Memecoin'], mode='lines', name='Memecoin')
trace_all_w = go.Scatter(x=df_temp_all.index, y=df_temp_all['All'], mode='lines', name='Market')

weekly_graph = go.Figure([trace_meme_w, trace_all_w])
weekly_graph.update_layout(title='Historical Weekly Returns',
                        xaxis_title='Date',
                        yaxis_title='Weekly Returns (%)',
                        yaxis=dict(range=[-50, 100]))
weekly_graph.show()